In [1]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.meandamage import MeanDamage

In [2]:
client = IncoreClient()

### Stert with building damage

We are going to run three chained analyses, Building damage, Mean damage and finally Building economic loss. The output of one analu=ysis is used as an input of the other. 

The example inventory is Seaside, OR.

In [3]:
from pyincore.analyses.buildingdamage import BuildingDamage
bldg_dmg = BuildingDamage(client)

In [4]:
# Seaside, OR building inventory
bldgs_id = "5bcf2fcbf242fe047ce79dad"
bldg_dmg.load_remote_input_dataset("buildings", bldgs_id)

# Load fragility mapping
fragility_service = FragilityService(client)
# Earthquake retrofitted building mappings
mapping_set = MappingSet(fragility_service.get_mapping("5d2789dbb9219c3c553c7977"))
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [5]:
# Seaside probabilistic earthquake 10000yr 
bldg_dmg.set_parameter("hazard_type", "earthquake")
bldg_dmg.set_parameter("hazard_id", "5ba8f379ec2309043520906f")
bldg_dmg.set_parameter("num_cpu", 1)

True

In [6]:
bldg_dmg.set_parameter("result_name", "bldg_dmg_seaside")

bldg_dmg.run_analysis()
building_damage_result = bldg_dmg.get_output_dataset("result")

### Mean damage

In [ ]:
The structural damage calculation uses mean damage and standard deviation values from damage ratios tables. 
The four damage state probabilities are multiplied by the mean damage and aggregated to get the Mean damage for 
each individual building. 

In [7]:
md = MeanDamage(client)

md.set_input_dataset("damage", building_damage_result)
# Damage ratio file
md.load_remote_input_dataset("dmg_ratios", "5a284f2ec7d30d13bc08209a")

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


True

In [ ]:
md.set_parameter("result_name", "bldg_mean_dmg_seaside")
md.set_parameter("damage_interval_keys", ["insignific", "moderate", "heavy", "complete"])
md.set_parameter("num_cpu", 1)

In [ ]:
md.run_analysis()
mean_damage_result = md.get_output_dataset('result')

### Building economic loss

In [ ]:
bldg_econ_dmg = BuildingEconLoss(client)
# Set input datasets
bldg_econ_dmg.set_input_dataset("damage", building_damage_result)

# Load Seaside building inventory from cache
bldg_econ_dmg.load_remote_input_dataset("buildings", bldgs_id)
# Get consumer price index
consumer_price_idx = "5f487cc852147a614c708039"
bldg_econ_dmg.load_remote_input_dataset("consumer_price_index", consumer_price_idx)

In [ ]:
# set parameters
bldg_econ_dmg.set_parameter("result_name", result_name)
bldg_econ_dmg.set_parameter("num_cpu", 1)

result_name = "seaside_bldg_econ_loss"

In [ ]:
# Run analysis and show results
bldg_econ_dmg.run_analysis()
bldg_econ_dmg.get_output_dataset('result').get_dataframe_from_csv()